https://dart-fss.readthedocs.io/en/latest/dart_fs.html

In [29]:
# !pip install dart_fss

In [36]:
import pandas as pd
import numpy as np
import requests
import json, traceback
from bs4 import BeautifulSoup as soup 
import re
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [1]:
import dart_fss as dart

# 초기설정

In [2]:
# Open DART API KEY 설정
api_key='e10b8471ac01a418f8888d36542cd39dc82bc330'
dart.set_api_key(api_key=api_key)

'e10b8471ac01a418f8888d36542cd39dc82bc330'

# 기업리스트출력

In [3]:
# 모든 상장된 기업 리스트 불러오기
corp_list= dart.get_corp_list()
corp_list

# 총 99054개의 공시된 기업 존재
# 하지만 재무제표가 존재하는 기업은 따로 존재

Output()

Output()

Output()

Number of companies: 99054

존재하는 기업들 단순 출력(기업코드는 주식코드와 구분됨)

In [4]:
corp_list.corps[:5]

[[00434003]다코,
 [00434456]일산약품,
 [00430964]굿앤엘에스,
 [00432403]한라판지,
 [00388953]크레디피아제이십오차유동화전문회사]

In [5]:
tmp_list = []

for i in corp_list.corps[:5]:
    print(str(i)[1:9])
    tmp_list.append(str(i)[1:9])

00434003
00434456
00430964
00432403
00388953


In [6]:
tmp_list

['00434003', '00434456', '00430964', '00432403', '00388953']

In [7]:
dart.corp.get_corp_list().corps[1]

Label,Data
corp_code,00434456
corp_name,일산약품
stock_code,None
modify_date,20170630


In [8]:
dart.corp.get_corp_list().corps[0]

Label,Data
corp_code,00434003
corp_name,다코
stock_code,None
modify_date,20170630


In [9]:
len(dart.corp.get_corp_list().corps)
for i in dart.corp.get_corp_list().corps[:10]:
    print(type(i))

<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>
<class 'dart_fss.corp.corp.Corp'>


산업 및 제품에 따른 출력

In [10]:
dart.corp.get_corp_list().sectors[:10]

['1차 비철금속 제조업',
 '1차 철강 제조업',
 '가구 제조업',
 '가전제품 및 정보통신장비 소매업',
 '가정용 기기 제조업',
 '가죽, 가방 및 유사제품 제조업',
 '개인 및 가정용품 임대업',
 '건물 건설업',
 '건물설비 설치 공사업',
 '건축기술, 엔지니어링 및 관련 기술 서비스업']

In [130]:
list = dart.corp.CorpList().find_by_sector('Y')
list
corp_list.find_by_product('반도체')[:10]

[[00301246]SFA반도체,
 [01068348]러셀,
 [00365457]제이티,
 [00197759]미래산업,
 [01489648]솔브레인,
 [00536286]윈팩,
 [01255652]레이크머티리얼즈,
 [00524421]테스,
 [00911229]라온테크,
 [00861100]아이케이세미콘]

# 재무정보 조회 및 엑셀 다운

In [12]:
# 삼성전자 code
corp_code = '00126380'

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

In [13]:
# 삼성전자
samsung = corp_list.find_by_corp_name(corp_code)

In [16]:
# 2012년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20200101')

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

### 연간, 반기, 분기보고서
str: 
‘annual’ 연간, 
‘half’ 연간 + 반기, 
‘quarter’ 연간 + 반기 + 분기 
list: [‘annual’] : 연간, [‘half’]: 반기, [‘quarter] 분기, [‘annual’, ‘half’]: 연간 + 반기

[‘annual’, ‘quarter’]: 연간 + 분기, [‘half’, ‘quarter’]: 반기 + 분기, [‘annual’, ‘half’, ‘quarter’]: 연간 + 반기 + 분기

In [29]:
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20200101', report_tp= 'quarter', lang = 'ko')

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/6 [00:00<?, ?report/s]

In [181]:
# 연결재무상태표
df_fs = fs['bs'] # 또는 df = fs[0] 또는 df = fs.show('bs')
# 연결재무상태표 추출에 사용된 Label 정보
labels_fs = fs.labels['bs']

# df_fs.info()
labels_fs

In [182]:
# 연결손익계산서(Income Statement)
df_is = fs['is'] # 또는 df = fs[1] 또는 df = fs.show('is')
# 연결손익계산서 추출에 사용된 Label 정보
labels_is = fs.labels['is']

# df_is

In [183]:
# 연결포괄손익계산서()
df_ci = fs['cis'] # 또는 df = fs[2] 또는 df = fs.show('cis')
# 연결포괄손익계산서 추출에 사용된 Label 정보
labels_ci = fs.labels['cis']

# 현금흐름표(Cash Flow)
df_cf = fs['cf'] # 또는 df = fs[3] 또는 df = fs.show('cf')
# 현금흐름표 추출에 사용된 Label 정보
labels_cf = fs.labels['cf']

In [30]:
# 재무제표 일괄저장 (default: 실행폴더/fsdata/{corp_code}_{report_tp}.xlsx)
fs.save()

# 재무제표 일괄저장
filename = '삼성전자_연간반기분기.xlsx'
path = './data'
fs.save(filename=filename, path=path)

'./data\\삼성전자_연간반기분기.xlsx'

### 회사 리스트 다운로드

In [39]:
# getStockCode() : 상장주식종목 코드번호를 html을 통해 데이터프레임 형태로 불러오는 함수
def getStockCode() -> pd.DataFrame:

    # 1. 상장지수 회사명 및 종목코드 데이터를 불러오기
    KOSPI_code_list = pd.read_html(html_stock_code + "?method=download&marketType=stockMkt")[0]
    KOSDAQ_code_list = pd.read_html(html_stock_code + "?method=download&marketType=kosdaqMkt")[0]
    
    # 각 시장별 회사명, 종목코드만 추출
    KOSPI_code_list = KOSPI_code_list[["회사명", "종목코드"]]
    KOSDAQ_code_list = KOSDAQ_code_list[["회사명", "종목코드"]]
    
    # 주식종목코드는 int 타입(연속형)으로 6자리에 맞추기 위한 0 추가하는 익명함수(lambda)
    KOSPI_code_list["종목코드"] = KOSPI_code_list["종목코드"].apply(lambda x: "0"*(6 - len(str(x))) + str(x))
    KOSDAQ_code_list["종목코드"] = KOSDAQ_code_list["종목코드"].apply(lambda x: "0"*(6 - len(str(x))) + str(x))
      

    # 2. KOSPI, KOSDAQ 상장주식종목에서 리츠와 ETF를 제거
    
    # 문자열 정규표현식에서 "리츠"와 "스팩"을 필터링하여 해당 행 번호를 추출하여 데이터프레임에서 삭제
    KOSPI_code_list.drop(
        KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
    KOSPI_code_list.drop(
        KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
    KOSDAQ_code_list.drop(
        KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
    KOSDAQ_code_list.drop(
        KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
    
      # [str.contains UserWarning] This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
    return KOSPI_code_list, KOSDAQ_code_list


html_stock_code = "https://kind.krx.co.kr/corpgeneral/corpList.do"
kospi, kosdaq = getStockCode()

C:\Users\jjgus\AppData\Local\Temp\ipykernel_8940\1046956579.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_8940\1046956579.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_8940\1046956579.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_8940\1046956

In [53]:
tmp_corps = kospi['회사명'].sample(n=5)

for i in tmp_corps:
    print(corp_list.find_by_corp_name(i, exactly=True)[0])

[00293716]명신산업
[00148540]CJ
[00113544]대한화섬
[00819596]웅진
[00146232]일성건설


In [55]:
# 기업명 검색
tmp_corp = corp_list.find_by_corp_name('대한화섬', exactly=True)[0]
tmp_corp

Label,Data
corp_code,00113544
corp_name,대한화섬
stock_code,003830
modify_date,20230109
sector,화학섬유 제조업
product,"화학섬유원사(FY,PTY,S.FIBER)제조"
corp_cls,Y


In [56]:
tmp_corps

440    명신산업
414      CJ
548    대한화섬
685      웅진
71     일성건설
Name: 회사명, dtype: object

In [191]:
corp_code = '00126380'
corp = corp_list.find_by_corp_name(corp_code)
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20200101')

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

https://dart-fss.readthedocs.io/en/latest/dart_fs.html#id2

In [ ]:
corp_code = '00126380'
corp = corp_list.find_by_corp_name(corp_code)
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20200101', report_tp='half', lang='ko')

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/3 [00:00<?, ?report/s]

# 현대차 재무제표 불러오기, 정제, 다운로드

In [57]:
dart.set_api_key(api_key = api_key)
corp_list = dart.get_corp_list()
code = corp_list.find_by_corp_name('현대자동차', exactly=True)[0]

fs = code.extract_fs(bgn_de='20210101')

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

In [58]:
fs.save()

'C:\\Users\\jjgus\\OneDrive - 123\\바탕 화면\\재무제표데이터소싱\\jupyter notebook\\fsdata\\00164742_annual.xlsx'

In [59]:
fs

{'bgn_de': '20210101',
 'corp_code': '00164742',
 'end_de': None,
 'financial statement': [{'title': '[D210000] Statement of financial position, '
                                   'current/non-current - Consolidated '
                                   'financial statements (Unit: KRW)'},
                         {'title': '[D310000] Income statement, by function of '
                                   'expense - Consolidated financial '
                                   'statements (Unit: KRW)'},
                         {'title': '[D410000] Statement of comprehensive '
                                   'income - Consolidated financial statements '
                                   '(Unit: KRW)'},
                         {'title': '[D520000] Statement of cash flows, '
                                   'indirect method - Consolidated financial '
                                   'statements (Unit: KRW)'}],
 'lang': 'ko',
 'report_tp': 'annual',
 'separate': False,
 'separator': True}

In [62]:
df_is=fs['is']
df_is.head()

[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)  \
                                                                                          concept_id   
0                                  ifrs-full_Revenue                                                   
1                              ifrs-full_CostOfSales                                                   
2                              ifrs-full_GrossProfit                                                   
3     dart_TotalSellingGeneralAdministrativeExpenses                                                   
4                           dart_OperatingIncomeLoss                                                   

                                                                            \
  label_ko                             label_en            class0   class1   
0      매출액                                Sales  손익계산서 [abstract]      매출액   
1     매출원가                        Cost of sales  손익계산서 [abstract]     매출원가   
2    매출총이익                         Gross profit  손익계산서 [abstract]    매출총이익   
3  판매비와관리비  Selling and administrative expenses  손익계산서 [abstract]  판매비와관리비   
4     영업이익               Operating income(loss)  손익계산서 [abstract]     영업이익   

                    20220101-20221231     20210101-20211231  \
  class2 class3             (연결재무제표,)             (연결재무제표,)   
0   None   None 142,527,535,000,000.0 117,610,626,000,000.0   
1   None   None 114,209,483,000,000.0  95,680,131,000,000.0   
2   None   None  28,318,052,000,000.0  21,930,495,000,000.0   
3   None   None  18,498,283,000,000.0  15,251,546,000,000.0   
4   None   None   9,819,769,000,000.0   6,678,949,000,000.0   

      20200101-20201231     20190101-20191231    20180101-20181231  \
              (연결재무제표,)             (연결재무제표,)            (연결재무제표,)   
0 103,997,601,000,000.0 105,746,422,000,000.0 96,812,609,000,000.0   
1  85,515,931,000,000.0  88,091,409,000,000.0 81,670,479,000,000.0   
2  18,481,670,000,000.0  17,655,013,000,000.0 15,142,130,000,000.0   
3  16,086,999,000,000.0  14,049,508,000,000.0 12,719,965,000,000.0   
4   2,394,671,000,000.0   3,605,505,000,000.0  2,422,165,000,000.0   

     20170101-20171231    20160101-20161231    20150101-20151231  \
             (연결재무제표,)            (연결재무제표,)            (연결재무제표,)   
0 96,376,079,000,000.0 93,649,024,000,000.0 91,958,736,000,000.0   
1 78,798,172,000,000.0 75,959,720,000,000.0 73,701,296,000,000.0   
2 17,577,907,000,000.0 17,689,304,000,000.0 18,257,440,000,000.0   
3 13,003,240,000,000.0 12,495,804,000,000.0 11,899,534,000,000.0   
4  4,574,667,000,000.0  5,193,500,000,000.0  6,357,906,000,000.0   

     20140101-20141231    20130101-20131231    20120101-20121231  
             (연결재무제표,)            (연결재무제표,)            (연결재무제표,)  
0 89,256,319,000,000.0 87,307,636,000,000.0 84,469,721,000,000.0  
1 70,126,276,000,000.0 67,859,491,000,000.0 64,967,273,000,000.0  
2 19,130,043,000,000.0 19,448,145,000,000.0 19,502,448,000,000.0  
3 11,580,057,000,000.0 11,132,648,000,000.0 11,061,847,000,000.0  
4  7,549,986,000,000.0  8,315,497,000,000.0  8,440,601,000,000.0

In [63]:
df_is['20200101-20201231'].tail()

,"(연결재무제표,)"
16,"5,454.0"
17,"5,502.0"
18,NaN
19,"5,454.0"
20,"5,502.0"


In [116]:
df_is.head()

[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)  \
                                                                                          concept_id   
0                                  ifrs-full_Revenue                                                   
1                              ifrs-full_CostOfSales                                                   
2                              ifrs-full_GrossProfit                                                   
3     dart_TotalSellingGeneralAdministrativeExpenses                                                   
4                           dart_OperatingIncomeLoss                                                   

                                                                            \
  label_ko                             label_en            class0   class1   
0      매출액                                Sales  손익계산서 [abstract]      매출액   
1     매출원가                        Cost of sales  손익계산서 [abstract]     매출원가   
2    매출총이익                         Gross profit  손익계산서 [abstract]    매출총이익   
3  판매비와관리비  Selling and administrative expenses  손익계산서 [abstract]  판매비와관리비   
4     영업이익               Operating income(loss)  손익계산서 [abstract]     영업이익   

                    20200101-20201231     20190101-20191231  \
  class2 class3             (연결재무제표,)             (연결재무제표,)   
0   None   None 103,997,601,000,000.0 105,746,422,000,000.0   
1   None   None  85,515,931,000,000.0  88,091,409,000,000.0   
2   None   None  18,481,670,000,000.0  17,655,013,000,000.0   
3   None   None  16,086,999,000,000.0  14,049,508,000,000.0   
4   None   None   2,394,671,000,000.0   3,605,505,000,000.0   

     20180101-20181231    20170101-20171231    20160101-20161231  \
             (연결재무제표,)            (연결재무제표,)            (연결재무제표,)   
0 96,812,609,000,000.0 96,376,079,000,000.0 93,649,024,000,000.0   
1 81,670,479,000,000.0 78,798,172,000,000.0 75,959,720,000,000.0   
2 15,142,130,000,000.0 17,577,907,000,000.0 17,689,304,000,000.0   
3 12,719,965,000,000.0 13,003,240,000,000.0 12,495,804,000,000.0   
4  2,422,165,000,000.0  4,574,667,000,000.0  5,193,500,000,000.0   

     20150101-20151231    20140101-20141231    20130101-20131231  
             (연결재무제표,)            (연결재무제표,)            (연결재무제표,)  
0 91,958,736,000,000.0 89,256,319,000,000.0 87,307,636,000,000.0  
1 73,701,296,000,000.0 70,126,276,000,000.0 67,859,491,000,000.0  
2 18,257,440,000,000.0 19,130,043,000,000.0 19,448,145,000,000.0  
3 11,899,534,000,000.0 11,580,057,000,000.0 11,132,648,000,000.0  
4  6,357,906,000,000.0  7,549,986,000,000.0  8,315,497,000,000.0

In [64]:
df_contents=df_is[['20200101-20201231','20190101-20191231','20180101-20181231','20170101-20171231','20160101-20161231']]

In [65]:
import pandas as pd

index = df_is['[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)']['label_ko']
df_index = pd.DataFrame(index)
df_contents = df_is[['20200101-20201231','20190101-20191231','20180101-20181231','20170101-20171231','20160101-20161231']].droplevel(1,axis=1)

income_statement = pd.merge(df_index, df_contents, left_index=True, right_index=True)

income_statement.columns=['Label','2020','2019','2018','2017','2016']

In [66]:
# df_index
IS = income_statement
IS

,Label,2020,2019,2018,2017,2016
0,매출액,"103,997,601,000,000.0","105,746,422,000,000.0","96,812,609,000,000.0","96,376,079,000,000.0","93,649,024,000,000.0"
1,매출원가,"85,515,931,000,000.0","88,091,409,000,000.0","81,670,479,000,000.0","78,798,172,000,000.0","75,959,720,000,000.0"
2,매출총이익,"18,481,670,000,000.0","17,655,013,000,000.0","15,142,130,000,000.0","17,577,907,000,000.0","17,689,304,000,000.0"
3,판매비와관리비,"16,086,999,000,000.0","14,049,508,000,000.0","12,719,965,000,000.0","13,003,240,000,000.0","12,495,804,000,000.0"
4,영업이익,"2,394,671,000,000.0","3,605,505,000,000.0","2,422,165,000,000.0","4,574,667,000,000.0","5,193,500,000,000.0"
5,공동기업및관계기업투자손익,"162,162,000,000.0","542,826,000,000.0","404,541,000,000.0","225,053,000,000.0","1,729,447,000,000.0"
6,금융수익,"813,916,000,000.0","827,120,000,000.0","823,499,000,000.0","972,943,000,000.0","1,111,238,000,000.0"
7,금융비용,"955,991,000,000.0","475,218,000,000.0","600,867,000,000.0","1,120,386,000,000.0","678,037,000,000.0"
8,기타수익,"1,308,642,000,000.0","1,120,958,000,000.0","967,281,000,000.0","1,153,744,000,000.0","1,177,887,000,000.0"
9,기타비용,"1,630,144,000,000.0","1,457,425,000,000.0","1,487,037,000,000.0","1,367,471,000,000.0","1,226,963,000,000.0"


In [67]:
IS.to_excel('hyundai_is.xlsx')

# Etc

In [ ]:

# Open DART API KEY 설정
api_key='e10b8471ac01a418f8888d36542cd39dc82bc330'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

# 삼성전자 검색
samsung = corp_list.find_by_corp_name('NAVER', exactly=True)[0]

# 2012년부터 연간 연결재무제표 불러오기
fs = samsung.extract_fs(bgn_de='20120101')

# 재무제표 검색 결과를 엑셀파일로 저장 ( 기본저장위치: 실행폴더/fsdata )
fs.save()

# dart_fss.api
https://dart-fss.readthedocs.io/en/latest/dart_api.html

In [68]:
import dart_fss as dart
import dart_fss

# Open DART API KEY 설정
api_key='e10b8471ac01a418f8888d36542cd39dc82bc330'
dart.set_api_key(api_key=api_key)

'e10b8471ac01a418f8888d36542cd39dc82bc330'

In [69]:
api_key = dart.get_api_key()

In [70]:
x = dart_fss.api.filings.get_corp_code()

Output()

In [71]:
x[2]['corp_name']

'굿앤엘에스'

In [73]:
len(x)

99054

In [74]:
x[:6]

[{'corp_code': '00434003',
  'corp_name': '다코',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00434456',
  'corp_name': '일산약품',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00430964',
  'corp_name': '굿앤엘에스',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00432403',
  'corp_name': '한라판지',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00388953',
  'corp_name': '크레디피아제이십오차유동화전문회사',
  'stock_code': None,
  'modify_date': '20170630'},
 {'corp_code': '00179984',
  'corp_name': '연방건설산업',
  'stock_code': None,
  'modify_date': '20170630'}]

In [78]:
# 삼성전자 code
corp_code = '00126380'

dart_fss.api.filings.search_filings(corp_code=corp_code)

NoDataReceived: 조회된 데이타가 없습니다.

In [75]:
dart_fss.api.filings.search_filings(corp_code='00366942', bgn_de=None, end_de=None, last_reprt_at='N', pblntf_ty=None, pblntf_detail_ty=None, corp_cls=None, 
                                    sort='date', sort_mth='desc', page_no=1, page_count=10)

NoDataReceived: 조회된 데이타가 없습니다.

In [79]:
corp_code = '00126380'
bsns_year = '2018'
reprt_code = '11013'

In [80]:
dart_fss.api.info.get_capital_increase(corp_code, bsns_year, reprt_code)

AttributeError: module 'dart_fss.api.info' has no attribute 'get_capital_increase'

In [82]:
dart_fss.api.info.indvdl_by_pay(corp_code, bsns_year, reprt_code)

NoDataReceived: 조회된 데이타가 없습니다.

In [44]:
dart_fss.api.info.get_investment(corp_code, bsns_year, reprt_code)

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20180515001699',
   'corp_cls': 'Y',
   'corp_code': '00126380',
   'corp_name': '삼성전자',
   'inv_prm': '-',
   'frst_acqs_de': '-',
   'invstmnt_purps': '-',
   'frst_acqs_amount': '-',
   'bsis_blce_qy': '-',
   'bsis_blce_qota_rt': '-',
   'bsis_blce_acntbk_amount': '-',
   'incrs_dcrs_acqs_dsps_qy': '-',
   'incrs_dcrs_acqs_dsps_amount': '-',
   'incrs_dcrs_evl_lstmn': '-',
   'trmend_blce_qy': '-',
   'trmend_blce_qota_rt': '-',
   'trmend_blce_acntbk_amount': '-',
   'recent_bsns_year_fnnr_sttus_tot_assets': '-',
   'recent_bsns_year_fnnr_sttus_thstrm_ntpf': '-'},
  {'rcept_no': '20180515001699',
   'corp_cls': 'Y',
   'corp_code': '00126380',
   'corp_name': '삼성전자',
   'inv_prm': '합계',
   'frst_acqs_de': '-',
   'invstmnt_purps': '-',
   'frst_acqs_amount': '-',
   'bsis_blce_qy': '-',
   'bsis_blce_qota_rt': '-',
   'bsis_blce_acntbk_amount': '-',
   'incrs_dcrs_acqs_dsps_qy': '-',
   'incrs_dcrs_acqs_dsps_amount': '-',

In [45]:
dart_fss.api.info.get_major_shareholder(corp_code, bsns_year, reprt_code)

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20180515001699',
   'corp_cls': 'Y',
   'corp_code': '00126380',
   'corp_name': '삼성전자',
   'stock_knd': '보통주',
   'rm': '-',
   'nm': '이건희',
   'relate': '본인',
   'bsis_posesn_stock_co': '4,985,464',
   'bsis_posesn_stock_qota_rt': '3.86',
   'trmend_posesn_stock_co': '4,985,464',
   'trmend_posesn_stock_qota_rt': '3.88'},
  {'rcept_no': '20180515001699',
   'corp_cls': 'Y',
   'corp_code': '00126380',
   'corp_name': '삼성전자',
   'stock_knd': '보통주',
   'rm': '장내매매 등',
   'nm': '삼성생명(특별계정)',
   'relate': '특수관계인',
   'bsis_posesn_stock_co': '466,715',
   'bsis_posesn_stock_qota_rt': '0.36',
   'trmend_posesn_stock_co': '453,675',
   'trmend_posesn_stock_qota_rt': '0.35'},
  {'rcept_no': '20180515001699',
   'corp_cls': 'Y',
   'corp_code': '00126380',
   'corp_name': '삼성전자',
   'stock_knd': '우선주',
   'rm': '장내매매 등',
   'nm': '삼성생명(특별계정)',
   'relate': '특수관계인',
   'bsis_posesn_stock_co': '30,778',
   'bsis_posesn_stock_qota_rt': 